In [1]:
import sys
import os
sys.path.append("../src")

import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import pylab as pl

from WSMBSS import *
from general_utils import *
from visualization_utils import * 

import warnings
warnings.filterwarnings("ignore")

notebook_name = 'Nonnegative_Antisparse_Copula'

In [2]:
@njit
def run_neural_dynamics_nnantisparse_jit(x_current, h, y, M_H, M_Y, 
                                        W_HX, W_YH, D1, D2, beta, zeta, neural_dynamic_iterations, 
                                        lr_start, lr_stop, OUTPUT_COMP_TOL,
                                        use_adam_opt = False, adam_beta1 = 0.9, 
                                         adam_beta2 = 0.999, adam_eps = 1e-8):

    if use_adam_opt:
        beta1 = adam_beta1
        beta2 = adam_beta2
        epsilon = adam_eps
        mt_h = np.zeros_like(h)
        vt_h = np.zeros_like(h)
        mt_y = np.zeros_like(y)
        vt_y = np.zeros_like(y)

        Gamma_H = np.diag(np.diag(M_H))
        M_hat_H = M_H - Gamma_H

        Gamma_Y = np.diag(np.diag(M_Y))
        M_hat_Y = M_Y - Gamma_Y

        v = ((1 - beta) * Gamma_H + beta * D1 @ Gamma_H @ D1) @ (h)
        u = Gamma_Y @ D2 @ (y)

        PreviousMembraneVoltages = {'v': np.zeros_like(v), 'u': np.zeros_like(u)}
        MembraneVoltageNotSettled = 1
        OutputCounter = 0

        while MembraneVoltageNotSettled & (OutputCounter < neural_dynamic_iterations):
            OutputCounter += 1
            MUV = max(lr_start/(1+OutputCounter*0.005), lr_stop)

            delv = -v + (1 - zeta) * beta * D1 @ W_HX @ x_current
            delv = delv - ((1 - zeta) * (1 - beta) * M_hat_H  + (1- zeta) * beta * D1 @ M_hat_H @ D1) @ h
            delv = delv + (1 - zeta) * (1 - beta) * W_YH.T @ D2 @ y

            mt_h = beta1 * mt_h + (1 - beta1)*delv
            vt_h = beta2 * vt_h + (1 - beta2)*delv**2

            mt_hat_h = mt_h / (1 - beta1**OutputCounter)
            vt_hat_h = vt_h / (1 - beta2**OutputCounter)

            v = v + MUV * mt_hat_h / (np.sqrt(vt_hat_h) + epsilon)

            h = v / np.diag(Gamma_H * ((1 - zeta) * (1 - beta) + (1 - zeta) * beta * D1 ** 2))

            delu = -u + W_YH @ h
            delu = delu - M_hat_Y @ D2 @ y

            mt_y = beta1 * mt_y + (1 - beta1)*delu
            vt_y = beta2 * vt_y + (1 - beta2)*delu**2

            mt_hat_y = mt_y / (1 - beta1**OutputCounter)
            vt_hat_y = vt_y / (1 - beta2**OutputCounter)
            u = u + MUV * mt_hat_y / (np.sqrt(vt_hat_y) + epsilon)

            y = u / np.diag(Gamma_Y * (D2))
            y = y*(y>=0)*(y<=1.0)+(y>1.0)*1.0

            MembraneVoltageNotSettled = 0
            if (np.linalg.norm(v - PreviousMembraneVoltages['v'])/(np.linalg.norm(v) + 1e-30) > OUTPUT_COMP_TOL) | (np.linalg.norm(u - PreviousMembraneVoltages['u'])/(np.linalg.norm(u) + 1e-30) > OUTPUT_COMP_TOL):
                MembraneVoltageNotSettled = 1
            PreviousMembraneVoltages['v'] = v
            PreviousMembraneVoltages['u'] = u
    else:
        Gamma_H = np.diag(np.diag(M_H))
        M_hat_H = M_H - Gamma_H

        Gamma_Y = np.diag(np.diag(M_Y))
        M_hat_Y = M_Y - Gamma_Y

        v = ((1 - beta) * Gamma_H + beta * D1 @ Gamma_H @ D1) @ (h)
        u = Gamma_Y @ D2 @ (y)

        PreviousMembraneVoltages = {'v': np.zeros_like(v), 'u': np.zeros_like(u)}
        MembraneVoltageNotSettled = 1
        OutputCounter = 0

        while MembraneVoltageNotSettled & (OutputCounter < neural_dynamic_iterations):
            OutputCounter += 1
            MUV = max(lr_start/(1+OutputCounter*0.005), lr_stop)

            delv = -v + (1 - zeta) * beta * D1 @ W_HX @ x_current
            delv = delv - ((1 - zeta) * (1 - beta) * M_hat_H  + (1- zeta) * beta * D1 @ M_hat_H @ D1) @ h
            delv = delv + (1 - zeta) * (1 - beta) * W_YH.T @ D2 @ y
            v = v + (MUV) * delv
            h = v / np.diag(Gamma_H * ((1 - zeta) * (1 - beta) + (1 - zeta) * beta * D1 ** 2))

            delu = -u + W_YH @ h
            delu = delu - M_hat_Y @ D2 @ y
            u = u + (MUV) * delu
            y = u / np.diag(Gamma_Y * (D2))
            y = y*(y>=0)*(y<=1.0)+(y>1.0)*1.0
            v = ((1 - beta) * Gamma_H + beta * D1 @ Gamma_H @ D1) @ (h)
            u = Gamma_Y @ D2 @ (y)
            MembraneVoltageNotSettled = 0
            if (np.linalg.norm(v - PreviousMembraneVoltages['v'])/(np.linalg.norm(v) + 1e-30) > OUTPUT_COMP_TOL) | (np.linalg.norm(u - PreviousMembraneVoltages['u'])/(np.linalg.norm(u) + 1e-30) > OUTPUT_COMP_TOL):
                MembraneVoltageNotSettled = 1
            PreviousMembraneVoltages['v'] = v
            PreviousMembraneVoltages['u'] = u

    return h,y, OutputCounter

def dlogdet(D, DEPS = 5e-8):
    d = np.diag(np.diag(D + DEPS * np.eye(len(D))) ** (-1))
    return d

In [3]:
np.random.seed(100)
rho = 0.5
N = 500000
NumberofSources = 5
NumberofMixtures = 10

# NoiseAmp = (10 ** (-SNR/20))# * np.sqrt(NumberofSources)

S = generate_correlated_copula_sources(rho = rho, df = 4, n_sources = NumberofSources, size_sources = N , 
                                       decreasing_correlation = True)

INPUT_STD = 0.28
A, X = WSM_Mixing_Scenario(S, NumberofMixtures, INPUT_STD)

SNR=30
X, NoisePart = addWGN(X, SNR, return_noise = True)

SNRinp = 10 * np.log10(np.sum(np.mean((X - NoisePart)**2, axis = 1)) / np.sum(np.mean(NoisePart**2, axis = 1)))
print("The following is the mixture matrix A")
display_matrix(A)
print("Input SNR is : {}".format(SNRinp))

The following is the mixture matrix A


<IPython.core.display.Math object>

Input SNR is : 29.99847169729263


In [4]:
if rho > 0.4:
    gamma_start = 0.05
    gamma_stop = 5*1e-4
else:
    gamma_start = 0.1
    gamma_stop = 1e-3
    
OUTPUT_COMP_TOL = 1e-7
MAX_OUT_ITERATIONS= 3000
LayerGains = [1,1]
LayerMinimumGains = [1e-3,1e-3]
LayerMaximumGains = [1e6,20]
WScalings = [0.0033,0.0033]
GamScalings = [2,1]
zeta = 1*1e-5
beta = 0.5
muD = [1, 1e-2]

neural_dynamic_iterations = 500
neural_lr_start = 0.75
neural_lr_stop = 0.05

s_dim = S.shape[0]
x_dim = X.shape[0]
h_dim = s_dim
samples = S.shape[1]
W_HX = np.eye(h_dim, x_dim)
W_YH = np.eye(s_dim, h_dim)

M_H = GamScalings[0] * np.eye(h_dim)
M_Y = GamScalings[1] * np.eye(s_dim)
D1 = LayerGains[0] * np.eye(h_dim)
D2 = LayerGains[1] * np.eye(s_dim)

H = np.zeros((h_dim,samples))
Y = np.zeros((s_dim,samples))

In [5]:
i_sample = 0
x_current = X[:, i_sample]
h = H[:, i_sample]
y = Y[:, i_sample]

h,y,oc = run_neural_dynamics_nnantisparse_jit(x_current, h, y, M_H, M_Y, 
                                    W_HX, W_YH, D1, D2, beta, zeta, neural_dynamic_iterations, 
                                    neural_lr_start, neural_lr_stop, OUTPUT_COMP_TOL)
h,y,oc

(array([ 0.07172777, -0.12295729, -0.22121189, -0.05889188, -0.1491297 ]),
 array([0.07172776, 0.        , 0.        , 0.        , 0.        ]),
 32)

In [6]:
MUS = np.max([gamma_start/(1+ np.log(1 + i_sample)/10),gamma_stop])

M_H = (1 - MUS) * M_H + MUS * np.outer(h,h)
W_HX = (1 - MUS) * W_HX + MUS * np.outer(h,x_current)

M_Y = (1 - MUS) * M_Y + MUS * np.outer(y,y)
W_YH = (1 - MUS) * W_YH + MUS * np.outer(y,h)

D1_prev = D1.copy()
D1derivative = (1 - zeta) * beta * np.diag(np.diag(M_H @ D1 @ M_H - W_HX @ W_HX.T)) + zeta * dlogdet(D1)
# D1secderivative = (1 - zeta) * beta * np.diag((np.diag(M_H)**2) * np.diag(D1)) + zeta * self.d2logdet(D1)
# D1 = D1 - muD[0] * D1derivative #/ D1secderivative
D1 = D1 - clipping(muD[0] * D1derivative, D1 * 1)

D2_prev = D2.copy()
D2derivative = (1 - zeta) * (1 - beta) * np.diag(np.diag(M_Y @ D2 @ M_Y - W_YH @ W_YH.T)) + zeta * dlogdet(D2)
# D2secderivative = (1 - zeta) * beta * np.diag((np.diag(M_Y) ** 2) * np.diag(D2)) + zeta * self.d2logdet(D2)
# D2 = D2 - muD[1] * D2derivative #/ D2secderivative
D2 = D2 - clipping(muD[1] * D2derivative, D2 * 1) 

d1 = np.diag(D1)
d2 = np.diag(D2)

D1 = np.diag(d1 * (d1 > LayerMinimumGains[0]) * (d1 < LayerMaximumGains[0]) + LayerMaximumGains[0] * (d1 >= LayerMaximumGains[0]) + LayerMinimumGains[0] * (d1 <= LayerMinimumGains[0]))
D2 = np.diag(d2 * (d2 > LayerMinimumGains[1]) * (d2 < LayerMaximumGains[1]) + LayerMaximumGains[1] * (d2 >= LayerMaximumGains[1]) + LayerMinimumGains[1] * (d2 <= LayerMinimumGains[1]))


In [11]:
W_HX

array([[ 9.50771725e-01, -1.76388070e-03, -3.17338975e-03,
        -8.44831988e-04, -2.13933638e-03,  3.72752063e-03,
         1.96565080e-03, -2.34092050e-03,  4.21593136e-04,
        -3.79115397e-03],
       [-1.32290822e-03,  9.53023682e-01,  5.43989324e-03,
         1.44822924e-03,  3.66729662e-03, -6.38979637e-03,
        -3.36956105e-03,  4.01285648e-03, -7.22704059e-04,
         6.49887800e-03],
       [-2.38003817e-03,  5.43989324e-03,  9.59786887e-01,
         2.60550264e-03,  6.59781667e-03, -1.14958536e-02,
        -6.06216196e-03,  7.21951184e-03, -1.30021359e-03,
         1.16921019e-02],
       [-6.33622889e-04,  1.44822924e-03,  2.60550264e-03,
         9.50693647e-01,  1.75649605e-03, -3.06047023e-03,
        -1.61389200e-03,  1.92200613e-03, -3.46147847e-04,
         3.11271619e-03],
       [-1.60449949e-03,  3.66729662e-03,  6.59781667e-03,
         1.75649605e-03,  9.54447909e-01, -7.74991404e-03,
        -4.08679823e-03,  4.86702407e-03, -8.76537218e-04,
         7.

In [8]:
D1

array([[0.001, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.001, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.001, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.001, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.001]])

In [9]:
D2

array([[0.99999991, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9999999 , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.9999999 , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.9999999 , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.9999999 ]])